# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "sw" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
# !mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
# os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [0]:
# !echo $gdrive_path

In [0]:
# Install opus-tools
# ! pip install opustools-pkg

In [2]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-sw.xml.gz not found. The following files are available for downloading:

   8 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-sw.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  94 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/sw.zip

 365 MB Total size
./JW300_latest_xml_en-sw.xml.gz ... 100% of 8 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_sw.zip ... 100% of 94 MB


In [4]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2019-10-30 04:02:19--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K   298KB/s    in 0.9s    

2019-10-30 04:02:21 (298 KB/s) - ‘test.en-any.en’ saved [277791/277791]

--2019-10-30 04:02:22--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-sw.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 

In [5]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [6]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 6478/979526 lines since contained in test set.


,source_sentence,target_sentence
0,“ Look !,“ Tazama !
1,I Am Making All Things New ”,Mimi Ninafanya Vitu Vyote Kuwa Vipya ”
2,The above is a promise from God that has been ...,Iliyopo juu ni ahadi itokayo kwa Mungu ambayo ...


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [8]:
import numpy as np
# drop duplicate translations
df_pp = df.drop_duplicates()

#drop empty lines (alp)
df_pp['source_sentence'].replace('', np.nan, inplace=True)
df_pp['target_sentence'].replace('', np.nan, inplace=True)
df_pp.dropna(subset=['source_sentence'], inplace=True)
df_pp.dropna(subset=['target_sentence'], inplace=True)

# drop conflicting translations
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/home/ks/.local/lib/python3.6/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/ks/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ks/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ks/anaconda3/lib/pytho

In [9]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)


# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
To make matters worse , I made the big mistake of opening my mouth to speak only when I was sure of my French .
Initially , the Waldenses withdrew from Roman Catholicism , the State religion in Central Europe .
The advantages of allowing more sunshine into a building have not gone unnoticed among retailers either .
Not even the joy from events such as Bangkok’s bicentennial celebration ​ — floral parades , historical exhibits , royal ceremonies , classical dances , and fireworks — ​ can blot out the sadness felt in this city .
Century Dictionary
The apostle Paul also warned against “ the trickery of men . ”
The predicted end of this diabolic system of things will not come before the Kingdom witness has been given in all the earth as a grand finale to the activity of the followers of Jesus Christ , the Witnesses of Jehovah .
Women as well as men are given many privileges by Jehovah .
As the study progressed , I began to see the need for making big changes in my personal



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [0]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [10]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

In [11]:
# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

In [12]:
# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

bpe.codes.4000	dev.en	     test.bpe.sw     test.sw	   train.en
dev.bpe.en	dev.sw	     test.en	     train.bpe.en  train.sw
dev.bpe.sw	test.bpe.en  test.en-any.en  train.bpe.sw


In [13]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
# ! cp train.* "$gdrive_path"
# ! cp test.* "$gdrive_path"
# ! cp dev.* "$gdrive_path"
# ! cp bpe.codes.4000 "$gdrive_path"
# ! ls "$gdrive_path"

cp: target '' is not a directory
cp: target '' is not a directory
cp: target '' is not a directory
cp: cannot create regular file '': No such file or directory
ls: cannot access '': No such file or directory


In [16]:
# Create that vocab using build_vocab
# ! sudo chmod 777 joeynmt/scripts/build_vocab.py
! /home/ks/code/tools/joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

In [17]:
# Some output
! echo "BPE Xhosa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

BPE Xhosa Sentences
N@@ g@@ ao kubwa ya imani ( T@@ azama f@@ ungu la 12 hadi 14 )
K@@ of@@ ia ya chu@@ ma ya wo@@ ko@@ vu ( T@@ azama f@@ ungu la 15 hadi 18 )
N@@ im@@ et@@ ambua kwamba watu hu@@ it@@ ikia vizuri wanapo@@ ona uki@@ zungum@@ zia habari za Biblia kwa sha@@ uku na unapo@@ fanya yote una@@ yo@@ weza kuwasaidia . ”
Up@@ anga wa roho ( T@@ azama f@@ ungu la 19 na 20 )
Kwa msaada wa Yehova tunaweza kus@@ im@@ ama im@@ ara na kum@@ p@@ inga !
Combined BPE Vocab
о@@
ḥ
е@@
с@@
и
х@@
і@@
д@@
і
➊


In [0]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [2]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_filtered_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "/home/ks/code/nlp/masakhane_sw2/joeynmt/data/{name}/train.bpe"
    dev:   "/home/ks/code/nlp/masakhane_sw2/joeynmt/data/{name}/dev.bpe"
    test:  "/home/ks/code/nlp/masakhane_sw2/joeynmt/data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/home/ks/code/nlp/masakhane_sw2/joeynmt/data/{name}/vocab.txt"
    trg_vocab: "/home/ks/code/nlp/masakhane_sw2/joeynmt/data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "/home/ks/code/tools/joeynmt/models/{name}_filtered_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path="/home/ks/code/nlp/masakhane_sw2/joeynmt", source_language=source_language, target_language=target_language)
with open("/home/ks/code/tools/joeynmt/configs/transformer_filtered_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [3]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train /home/ks/code/tools/joeynmt/configs/transformer_filtered_$src$tgt.yaml

/home/ks/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ks/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ks/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ks/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dty

2019-10-30 11:10:00,042 cfg.name                           : ensw_filtered_transformer
2019-10-30 11:10:00,042 cfg.data.src                       : en
2019-10-30 11:10:00,042 cfg.data.trg                       : sw
2019-10-30 11:10:00,042 cfg.data.train                     : /home/ks/code/nlp/masakhane_sw2/joeynmt/data/ensw/train.bpe
2019-10-30 11:10:00,042 cfg.data.dev                       : /home/ks/code/nlp/masakhane_sw2/joeynmt/data/ensw/dev.bpe
2019-10-30 11:10:00,042 cfg.data.test                      : /home/ks/code/nlp/masakhane_sw2/joeynmt/data/ensw/test.bpe
2019-10-30 11:10:00,042 cfg.data.level                     : bpe
2019-10-30 11:10:00,042 cfg.data.lowercase                 : False
2019-10-30 11:10:00,042 cfg.data.max_sent_length           : 100
2019-10-30 11:10:00,042 cfg.data.src_vocab                 : /home/ks/code/nlp/masakhane_sw2/joeynmt/data/ensw/vocab.txt
2019-10-30 11:10:00,042 cfg.data.trg_vocab                 : /home/ks/code/nlp/masakhane_sw2/joeynmt/data/e

2019-10-30 11:13:49,033 Epoch   1 Step:     1100 Batch Loss:     4.341217 Tokens per Sec:    17921, Lr: 0.000300
2019-10-30 11:14:03,124 Epoch   1 Step:     1200 Batch Loss:     4.113015 Tokens per Sec:    17886, Lr: 0.000300
2019-10-30 11:14:17,278 Epoch   1 Step:     1300 Batch Loss:     4.046868 Tokens per Sec:    18521, Lr: 0.000300
2019-10-30 11:14:31,368 Epoch   1 Step:     1400 Batch Loss:     4.203510 Tokens per Sec:    18247, Lr: 0.000300
2019-10-30 11:14:45,529 Epoch   1 Step:     1500 Batch Loss:     4.009446 Tokens per Sec:    18183, Lr: 0.000300
2019-10-30 11:14:59,485 Epoch   1 Step:     1600 Batch Loss:     4.068118 Tokens per Sec:    18034, Lr: 0.000300
2019-10-30 11:15:13,727 Epoch   1 Step:     1700 Batch Loss:     3.987222 Tokens per Sec:    18242, Lr: 0.000300
2019-10-30 11:15:27,974 Epoch   1 Step:     1800 Batch Loss:     3.617097 Tokens per Sec:    18360, Lr: 0.000300
2019-10-30 11:15:42,104 Epoch   1 Step:     1900 Batch Loss:     3.629297 Tokens per Sec:    178

2019-10-30 11:24:26,137 Epoch   1 Step:     4600 Batch Loss:     2.670850 Tokens per Sec:    17279, Lr: 0.000300
2019-10-30 11:24:40,383 Epoch   1 Step:     4700 Batch Loss:     2.753960 Tokens per Sec:    17992, Lr: 0.000300
2019-10-30 11:24:54,637 Epoch   1 Step:     4800 Batch Loss:     3.159465 Tokens per Sec:    18338, Lr: 0.000300
2019-10-30 11:25:09,078 Epoch   1 Step:     4900 Batch Loss:     2.992566 Tokens per Sec:    17680, Lr: 0.000300
2019-10-30 11:25:23,598 Epoch   1 Step:     5000 Batch Loss:     3.456820 Tokens per Sec:    17614, Lr: 0.000300
2019-10-30 11:26:06,438 Hooray! New best validation result [ppl]!
2019-10-30 11:26:06,438 Saving new checkpoint.
2019-10-30 11:26:06,567 Example #0
2019-10-30 11:26:06,567 	Source:     She became pregnant at the age of 17 .
2019-10-30 11:26:06,567 	Reference:  Alipata mimba akiwa na umri wa miaka 17 .
2019-10-30 11:26:06,567 	Hypothesis: Alikuwa amekuwa na umri wa miaka 17 .
2019-10-30 11:26:06,567 Example #1
2019-10-30 11:26:06,56

2019-10-30 11:35:51,320 Hooray! New best validation result [ppl]!
2019-10-30 11:35:51,320 Saving new checkpoint.
2019-10-30 11:35:51,861 Example #0
2019-10-30 11:35:51,862 	Source:     She became pregnant at the age of 17 .
2019-10-30 11:35:51,862 	Reference:  Alipata mimba akiwa na umri wa miaka 17 .
2019-10-30 11:35:51,862 	Hypothesis: Alikuwa akitafuta umri wa miaka 17 .
2019-10-30 11:35:51,862 Example #1
2019-10-30 11:35:51,863 	Source:     The Bible does not promote credulity .
2019-10-30 11:35:51,863 	Reference:  Biblia haipendekezi wepesi wa kuamini bila ushuhuda .
2019-10-30 11:35:51,864 	Hypothesis: Biblia haitegemei uumbaji wa uumbaji .
2019-10-30 11:35:51,864 Example #2
2019-10-30 11:35:51,864 	Source:     What are some things that cause anxiety , and what can anxiety result in ?
2019-10-30 11:35:51,865 	Reference:  Ni baadhi ya mambo gani yanayosababisha hangaiko , na hangaiko laweza kutokeza nini ?
2019-10-30 11:35:51,865 	Hypothesis: Ni mambo gani yanayosababisha mahangai

...

2019-10-31 01:34:50,191 Epoch  29 Step:   267100 Batch Loss:     1.184764 Tokens per Sec:    17038, Lr: 0.000050
2019-10-31 01:35:05,207 Epoch  29 Step:   267200 Batch Loss:     1.308176 Tokens per Sec:    17386, Lr: 0.000050
2019-10-31 01:35:20,242 Epoch  29 Step:   267300 Batch Loss:     1.151002 Tokens per Sec:    17322, Lr: 0.000050
2019-10-31 01:35:35,139 Epoch  29 Step:   267400 Batch Loss:     1.158971 Tokens per Sec:    17103, Lr: 0.000050
2019-10-31 01:35:50,313 Epoch  29 Step:   267500 Batch Loss:     1.223896 Tokens per Sec:    17345, Lr: 0.000050
2019-10-31 01:36:05,310 Epoch  29 Step:   267600 Batch Loss:     1.084948 Tokens per Sec:    16946, Lr: 0.000050
2019-10-31 01:36:20,408 Epoch  29 Step:   267700 Batch Loss:     1.135738 Tokens per Sec:    16869, Lr: 0.000050
2019-10-31 01:36:35,622 Epoch  29 Step:   267800 Batch Loss:     1.212309 Tokens per Sec:    16869, Lr: 0.000050
2019-10-31 01:36:50,761 Epoch  29 Step:   267900 Batch Loss:     1.201303 Tokens per Sec:    170

2019-10-31 01:46:21,791 Epoch  29 Step:   270800 Batch Loss:     1.213597 Tokens per Sec:    17053, Lr: 0.000050
2019-10-31 01:46:36,816 Epoch  29 Step:   270900 Batch Loss:     1.157011 Tokens per Sec:    17241, Lr: 0.000050
2019-10-31 01:46:51,923 Epoch  29 Step:   271000 Batch Loss:     1.211065 Tokens per Sec:    17165, Lr: 0.000050
2019-10-31 01:47:36,446 Example #0
2019-10-31 01:47:36,447 	Source:     She became pregnant at the age of 17 .
2019-10-31 01:47:36,447 	Reference:  Alipata mimba akiwa na umri wa miaka 17 .
2019-10-31 01:47:36,447 	Hypothesis: Alikuwa na mimba alipokuwa na umri wa miaka 17 .
2019-10-31 01:47:36,447 Example #1
2019-10-31 01:47:36,447 	Source:     The Bible does not promote credulity .
2019-10-31 01:47:36,447 	Reference:  Biblia haipendekezi wepesi wa kuamini bila ushuhuda .
2019-10-31 01:47:36,447 	Hypothesis: Biblia haisemi kwamba mtu ana sifa ya kukubali sababu .
2019-10-31 01:47:36,447 Example #2
2019-10-31 01:47:36,447 	Source:     What are some thin

2019-10-31 01:57:35,851 Epoch  30 Step:   274100 Batch Loss:     1.123149 Tokens per Sec:    17368, Lr: 0.000050
2019-10-31 01:57:50,773 Epoch  30 Step:   274200 Batch Loss:     1.181982 Tokens per Sec:    17041, Lr: 0.000050
2019-10-31 01:58:05,702 Epoch  30 Step:   274300 Batch Loss:     1.082892 Tokens per Sec:    17289, Lr: 0.000050
2019-10-31 01:58:20,823 Epoch  30 Step:   274400 Batch Loss:     1.534480 Tokens per Sec:    16789, Lr: 0.000050
2019-10-31 01:58:35,894 Epoch  30 Step:   274500 Batch Loss:     1.086880 Tokens per Sec:    17583, Lr: 0.000050
2019-10-31 01:58:50,909 Epoch  30 Step:   274600 Batch Loss:     1.237480 Tokens per Sec:    17248, Lr: 0.000050
2019-10-31 01:59:05,919 Epoch  30 Step:   274700 Batch Loss:     1.206151 Tokens per Sec:    17098, Lr: 0.000050
2019-10-31 01:59:21,118 Epoch  30 Step:   274800 Batch Loss:     1.249125 Tokens per Sec:    17052, Lr: 0.000050
2019-10-31 01:59:36,065 Epoch  30 Step:   274900 Batch Loss:     1.210787 Tokens per Sec:    171

2019-10-31 02:09:02,272 Epoch  30 Step:   277800 Batch Loss:     1.105389 Tokens per Sec:    17456, Lr: 0.000035
2019-10-31 02:09:17,075 Epoch  30 Step:   277900 Batch Loss:     1.212099 Tokens per Sec:    17489, Lr: 0.000035
2019-10-31 02:09:32,138 Epoch  30 Step:   278000 Batch Loss:     1.091322 Tokens per Sec:    16919, Lr: 0.000035
2019-10-31 02:10:16,195 Example #0
2019-10-31 02:10:16,195 	Source:     She became pregnant at the age of 17 .
2019-10-31 02:10:16,195 	Reference:  Alipata mimba akiwa na umri wa miaka 17 .
2019-10-31 02:10:16,195 	Hypothesis: Alikuwa na mimba alipokuwa na umri wa miaka 17 .
2019-10-31 02:10:16,195 Example #1
2019-10-31 02:10:16,195 	Source:     The Bible does not promote credulity .
2019-10-31 02:10:16,195 	Reference:  Biblia haipendekezi wepesi wa kuamini bila ushuhuda .
2019-10-31 02:10:16,195 	Hypothesis: Biblia haisemi kwamba mtu ana sifa ya kukubali sababu .
2019-10-31 02:10:16,195 Example #2
2019-10-31 02:10:16,196 	Source:     What are some thin

2019-10-31 02:20:09,446 Epoch  30 Step:   281100 Batch Loss:     1.140924 Tokens per Sec:    17191, Lr: 0.000035
2019-10-31 02:20:24,354 Epoch  30 Step:   281200 Batch Loss:     1.061550 Tokens per Sec:    17530, Lr: 0.000035
2019-10-31 02:20:39,233 Epoch  30 Step:   281300 Batch Loss:     1.142339 Tokens per Sec:    17272, Lr: 0.000035
2019-10-31 02:20:54,372 Epoch  30 Step:   281400 Batch Loss:     1.203827 Tokens per Sec:    16785, Lr: 0.000035
2019-10-31 02:21:09,555 Epoch  30 Step:   281500 Batch Loss:     1.136458 Tokens per Sec:    17312, Lr: 0.000035
2019-10-31 02:21:17,507 Epoch  30: total training loss 10767.81
2019-10-31 02:21:17,508 Training ended after  30 epochs.
2019-10-31 02:21:17,508 Best validation result at step   279000:   2.80 ppl.
Traceback (most recent call last):
  File "/home/ks/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ks/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, ru

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [3]:
# Output our validation accuracy
! cat "/home/ks/code/tools/joeynmt/models/${src}${tgt}_filtered_transformer/validations.txt"

Steps: 1000	Loss: 120892.71094	PPL: 81.34162	bleu: 0.87183	LR: 0.00030000	*
Steps: 2000	Loss: 99126.69531	PPL: 36.84446	bleu: 2.80529	LR: 0.00030000	*
Steps: 3000	Loss: 85728.57031	PPL: 22.62866	bleu: 5.11622	LR: 0.00030000	*
Steps: 4000	Loss: 77592.10938	PPL: 16.83018	bleu: 8.42625	LR: 0.00030000	*
Steps: 5000	Loss: 71383.70312	PPL: 13.42720	bleu: 10.88299	LR: 0.00030000	*
Steps: 6000	Loss: 67318.64062	PPL: 11.58111	bleu: 12.83706	LR: 0.00030000	*
Steps: 7000	Loss: 63303.48828	PPL: 10.00700	bleu: 15.14659	LR: 0.00030000	*
Steps: 8000	Loss: 60214.65625	PPL: 8.94324	bleu: 16.16055	LR: 0.00030000	*
Steps: 9000	Loss: 57998.01562	PPL: 8.25027	bleu: 17.94183	LR: 0.00030000	*
Steps: 10000	Loss: 55727.73438	PPL: 7.59616	bleu: 18.76534	LR: 0.00030000	*
Steps: 11000	Loss: 54107.03125	PPL: 7.16117	bleu: 20.55164	LR: 0.00030000	*
Steps: 12000	Loss: 52610.32812	PPL: 6.78162	bleu: 21.35097	LR: 0.00030000	*
Steps: 13000	Loss: 50830.55469	PPL: 6.35638	bleu: 22.80579	LR: 0.00030000	*
Steps: 14000	Loss

In [6]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "/home/ks/code/tools/joeynmt/models/${src}${tgt}_filtered_transformer/config.yaml"

/home/ks/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ks/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ks/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ks/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dty